In [ ]:
import pandas as pd
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import plotly.graph_objects as go
import json

In [ ]:
# Read in Brazil fire data
brazil_fire = Path('resources/brazil_fires_by_state.csv')
brazil_df = pd.read_csv(brazil_fire, encoding='ISO-8859-1')

# find unique state name for proper abbrevations
states = brazil_df['state']
states.unique()


In [ ]:
# rename states to 2-letter code so that data can merge
state_rename_mapping = {
    'Acre' : "AC",
    'Alagoas' : "AL",
    'Amapa' : "AP", 
    'Amazonas' : "AM", 
    'Bahia' : "BA", 
    'Ceara' : "CE",
    'Distrito Federal' : "DF", 
    'Espirito Santo' : "ES", 
    'Goias' : "GO", 
    'Maranhao' : "MA",
    'Mato Grosso' : "MT", 
    'Minas Gerais' : "MG", 
    'Pará' : "PA", 
    'Paraiba' :"PB", 
    'Pernambuco' : "PE",
    'Piau' : "PI",
    'Rio' : "RJ", 
    'Rondonia' : "RO",
    'Roraima' : "RR", 
    'Santa Catarina' : "SC",
    'Sao Paulo' : "SP", 
    'Sergipe' : "SE", 
    'Tocantins' : "TO"
}

In [ ]:
# rename state column values with abbrevations for each
brazil_df['state'] = brazil_df['state'].replace(state_rename_mapping)
brazil_df.head()

In [ ]:
# rename column label
brazil_abbrev = brazil_df.rename(columns = {
 "state" : "state_abbr"
})

brazil_abbrev.head()

In [ ]:
# group by year and state to find total reported fires for 2013
brazil_fire_2013 = brazil_abbrev[brazil_abbrev['year'] == 2013]
group_2013 = brazil_fire_2013.groupby(['year', 'state_abbr'])['number'].sum().reset_index()

group_2013.head()

In [ ]:
# repeat above for 2017
brazil_fire_2017 = brazil_abbrev[brazil_abbrev['year'] == 2017]
group_2017 = brazil_fire_2017.groupby(['year', 'state_abbr'])['number'].sum().reset_index()

group_2017.head()

In [ ]:
# load json file with state geometry
with open('resources/brazil_geo.json') as f:
    json_data = json.load(f)

# Convert to GeoDataFrame with GeoPandas
gdf = gpd.GeoDataFrame.from_features(json_data['features'])

gdf.head()

In [ ]:
# create state abbreviations to use in json file
state_abbr = {
    "Acre": "AC",
    "Alagoas": "AL",
    "Amapá": "AP",
    "Amazonas": "AM",
    "Bahia": "BA",
    "Ceará": "CE",
    "Espírito Santo": "ES",
    "Goiás": "GO",
    "Maranhão": "MA",
    "Mato Grosso": "MT",
    "Mato Grosso do Sul": "MS",
    "Minas Gerais": "MG",
    "Pará": "PA",
    "Paraíba": "PB",
    "Paraná": "PR",
    "Pernambuco": "PE",
    "Piauí": "PI",
    "Rio de Janeiro": "RJ",
    "Rio Grande do Norte": "RN",
    "Rio Grande do Sul": "RS",
    "Rondônia": "RO",
    "Roraima": "RR",
    "São Paulo": "SP",
    "Sergipe": "SE",
    "Tocantins": "TO"
}

In [ ]:
# add abbreviations to the json file
gdf['state_abbr'] = gdf['name'].map(state_abbr)

In [ ]:
# merge 2013 and 2017 data with the json file
gdf_2013 = gdf.merge(brazil_fire_2013, how='left', on='state_abbr')
gdf_2017 = gdf.merge(brazil_fire_2017, how='left', on='state_abbr')

gdf_2013.head()

In [ ]:
# Make 2013 Choropleth map
fig_2013 = px.choropleth(
    gdf_2013,
    geojson=gdf_2013.geometry,
    locations=gdf_2013.index,
    color='number',
    hover_name='name',
    hover_data=['number'],
    color_continuous_scale="Reds",
    title="Wildfires in Brazil - 2013", 
    height = 500, 
    width = 900
)
fig_2013.update_geos(fitbounds="locations", visible=False) 

# fig_2013.write_html("brazil_2013.html")
fig_2013.show()

In [ ]:

# Make 2017 Choropleth map
fig_2017 = px.choropleth(
    gdf_2017,
    geojson=gdf_2017.geometry,
    locations=gdf_2017.index,
    color='number',
    hover_name='name',
    hover_data=['number'],
    color_continuous_scale="Reds",
    title="Wildfires in Brazil - 2017", 
    height = 500, 
    width = 900
)
fig_2017.update_geos(fitbounds="locations", visible=False) 

# fig_2017.write_html("brazil_2017.html")
fig_2017.show()